### Installation
For this fine tuning process we utilize and make changes to the unsloth LLaMa 3.1 8B parmeter model found on [github](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-Alpaca.ipynb#scrollTo=QmUBVEnvCDJv). The main changes include adding the perplexity evaluation, changing the dataset used by the model, and creating a custom prompt for the model. In this section we load our dependencies comments go into further detail as to the process.

In [ ]:
# %%capture # Keep this if you want to suppress pip output
import os
# Check if not in Colab before running pip install
if "COLAB_" not in "".join(os.environ.keys()):
     print("Running outside Colab. Using: pip install unsloth bitsandbytes accelerate xformers peft trl sentencepiece protobuf datasets huggingface_hub hf_transfer")
     !pip install unsloth bitsandbytes accelerate xformers peft trl sentencepiece protobuf datasets huggingface_hub hf_transfer
else:
     # Do this only in Colab notebooks! Otherwise use pip install unsloth
     print("Running in Colab. Using specific versions for compatibility.")
     !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
     !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
     !pip install --no-deps unsloth

Running in Colab. Using specific versions for compatibility.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully unins

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset, DatasetDict
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from unsloth import is_bfloat16_supported
import math # Import math for perplexity calculation

# Configuration
max_seq_length = 2048
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


selected_model_name = "unsloth/Meta-Llama-3.1-8B"
print(f"Loading model: {selected_model_name}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = selected_model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

We now add LoRA adapters so we only need to update 1 to 10% of all parameters.

In [ ]:
print("Model loaded. Preparing PEFT model...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)
print("PEFT model prepared.")

Model loaded. Preparing PEFT model...
PEFT model prepared.


<a name="Data"></a>
### Data Prep
We utilize a [chemistry dataset](https://huggingface.co/datasets/mlfoundations-dev/stackexchange_chemistry) with general knowledge on the field of chemistry to help train our model. With this we are able to hone in on specific knowledge in chemistry to help produce a more knowledgable and accurate output. We also split our dataset into a train and eval category.

In [ ]:
# Configuration
max_seq_length = 2048
dtype = None
load_in_4bit = True

# --- Data Preparation ---
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:
{output}"""

EOS_TOKEN = tokenizer.eos_token
if EOS_TOKEN is None:
    print("Warning: EOS_TOKEN is None. Adding a default one: '<|end_of_text|>'.")
    tokenizer.add_special_tokens({'eos_token': '<|end_of_text|>'})
    EOS_TOKEN = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

# Load the raw dataset split
dataset = load_dataset("mlfoundations-dev/stackexchange_chemistry", split="train")

# --- Split the dataset BEFORE formatting ---
dataset_split = dataset.train_test_split(test_size=0.1, seed=42)  # 90% train, 10% eval
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print("Dataset loaded and split.")
print(f"Training examples: {len(train_dataset)}")
print(f"Evaluation examples: {len(eval_dataset)}")

def formatting_prompts_func(example):
    instructions = example["instruction"]
    inputs       = "" * len(instructions) #Here we are setting each position for the conversations column to null in a sense since we dont want to use it in training or our output
    outputs      = example["completion"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

NameError: name 'tokenizer' is not defined

<a name="Train"></a>
### Train the model
Now we use Huggingface TRL's `SFTTrainer`. More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We decided to set the number of epochs to 1 for our model due to the length of time. In this section we also setup the evaluation for the perplexity which is displayed in the model output.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = None,
    formatting_func=formatting_prompts_func,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = None,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

# --- Start Training (includes evaluation steps) ---
print("Starting training...")
trainer_stats = trainer.train()
print("Training finished.")

# --- Evaluation Results ---
# The trainer automatically calculates eval loss during training steps defined by eval_steps.
# Perplexity is calculated as exp(eval_loss). This will be printed in the logs during training.
# You can also explicitly run evaluate after training:
print("Running final evaluation on the best checkpoint...")
eval_results = trainer.evaluate()
print(f"Final Evaluation Results: {eval_results}")

# Calculate and print perplexity from final evaluation results
eval_loss = eval_results.get("eval_loss")
if eval_loss is not None:
    # Use math.exp for standard Python float
    perplexity = math.exp(eval_loss)
    print(f"Final Perplexity: {perplexity:.4f}")
else:
    print("Eval loss not found in evaluation results.")

KeyError: 'instruction'

#Inference

In [ ]:
# --- Inference Example (after training) ---
print("\n--- Inference Example ---")
# Ensure model is in inference mode after training & potential loading of best model
FastLanguageModel.for_inference(model) # Prepare model for faster inference

# Use the same prompt structure
alpaca_prompt_inference = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}""" # The {} for Response is where the model starts generating

# Prepare input for inference
instruction_text = """Write up a research approach around 500-600 words long on the topic of using NLP to identify properties and functions of molecules.
Share relevant information that would help solve this. Do not use any numbering or bullet points in your model output. Write text in full sentences and paragraphs.
Be verbose and give some examples of how we can be given an input, what that input would be, and how the output would be produced and formatted."""

inputs = tokenizer(
[
    alpaca_prompt_inference.format(
        instruction_text, # instruction
        "",               # input text is left blank as we never utilize its column
        "",               # Output left blank for generation
    )
], return_tensors = "pt").to(model.device) # Ensure tensor is on the same device as the model

text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True) # Skip prompt during streaming

print(f"\nGenerating response for instruction: '{instruction_text[:100]}...'")
_ = model.generate(**inputs,
                   streamer = text_streamer,
                   eos_token_id = tokenizer.eos_token_id, # Ensure generation stops at EOS
                   pad_token_id=tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id # Set pad token ID
                   )

print("\nInference complete.")



--- Inference Example ---

Generating response for instruction: 'Write up a research approach around 500-600 words long on the topic of using NLP to identify propert...'
NLP is a field of study that focuses on the interactions between humans and computers, specifically in the context of natural language processing. In this case, we are interested in using NLP to identify properties and functions of molecules. One approach to solving this problem could be to use a neural network architecture called a recurrent neural network (RNN). RNNs are well suited to this task because they can process sequential data such as sentences or paragraphs. Another possible approach would be to use a convolutional neural network (CNN). CNNs are good at recognizing patterns in images or videos, but they can also be used to analyze text data. Finally, we could try a combination of both RNNs and CNNs, which is known as a hybrid model. The choice of model will depend on the specific properties and functions w